In [3]:
import tensorflow as tf
import numpy as np

In [2]:
# Wy = Wh 동일하다고 가정한 구조!

n_inputs = 3
n_neurons = 5

# 입력값 정의
X0 = tf.placeholder(tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(tf.float32, shape=[None, n_inputs])

# Variable(가변적인 값들)
# 입력값 -> RNN cell로 들어가는 Weight 정의
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons]))
# RNN cell -> 중간출력값으로 진행될 때 Weight 정의
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons]))
# bias 정의 - 더하기 때문에 1차원, 2차원이어도 상관없음..하지만 직관 위해 2차원으로!
b = tf.Variable(tf.zeros(shape=[1, n_neurons]))

# 첫 번째 RNN cell에서 나온 중간출력값 정의
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
# 첫 번쨰 RNN cell에서 나온 state(Wh)를 Wy라고 가정하고 합해서 두번째 중간출력값 정의
# tf.matmul(Y0, Wy) => 4 by 5 
# tf.matmul(X1, Wx) + b => 4 by 5
Y1 = tf.tanh(tf.matmul(Y0, Wy)+ tf.matmul(X1, Wx) + b)

# Sample이 4개인 batch
X0_batch = np.array([[0,1,2],
                    [3,4,5],
                    [6,7,8],
                    [9,0,1]])
X1_batch = np.array([[9,8,7],
                    [3,4,5],
                    [6,5,4],
                    [3,2,1]])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    Y0_val, Y1_val = sess.run([Y0, Y1],
                             feed_dict={X0: X0_batch,
                                       X1: X1_batch})
    print("중간출력 Y0 shape:",Y0_val.shape,'\n',Y0_val)
    print("중간출력 Y1 shape:",Y1_val.shape,'\n',Y1_val)

중간출력 Y0 shape: (4, 5) 
 [[ 0.8353981  -0.8715248  -0.85353863 -0.99378526  0.9562326 ]
 [ 0.8288415  -0.9999783  -0.99999887 -1.          1.        ]
 [ 0.8220491  -1.         -1.         -1.          1.        ]
 [-1.         -0.9999639  -1.         -1.          0.9999998 ]]
중간출력 Y1 shape: (4, 5) 
 [[-0.8724218  -1.         -1.         -1.          1.        ]
 [ 0.797112   -1.         -0.9999998  -1.          0.9999989 ]
 [-0.8720081  -1.         -1.         -1.          1.        ]
 [-0.8052956  -0.99998325 -0.9999951  -0.99999845  0.9999768 ]]


In [9]:
# 위 구조를 RNN 관련 함수로 간단하게 구현!
tf.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, shape=[None, n_inputs])
X1 = tf.placeholder(tf.float32, shape=[None, n_inputs])

# RNN cell 정의 - cell안의 노드 개수 설정해주기
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# 위에서 만든 RNN cell time step만큼 만들어주고 연결하기
outputs, states = tf.nn.static_rnn(cell=basic_cell,
                                  inputs=[X0, X1],
                                  dtype=tf.float32)
# 중간 출력값 할당
Y0, Y1 = outputs

X0_batch = np.array([[0,1,2],
                    [3,4,5],
                    [6,7,8],
                    [9,0,1]])
X1_batch = np.array([[9,8,7],
                    [3,4,5],
                    [6,5,4],
                    [3,2,1]])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    Y0_val, Y1_val = sess.run([Y0, Y1],
                             feed_dict={X0: X0_batch,
                                       X1: X1_batch})
    print("Y0_val shape:",Y0_val.shape,'\n',Y0_val,'\n')
    print("Y1_val shape:",Y1_val.shape,'\n',Y1_val)

Y0_val shape: (4, 5) 
 [[-0.93513733  0.813686   -0.61397874  0.75377196 -0.8495282 ]
 [-0.99995434  0.99945503 -0.44305488  0.99651104 -0.97651815]
 [-1.          0.9999985  -0.23244105  0.9999565  -0.99653614]
 [-0.86476994  0.9997755   0.9994912   0.840745    0.99782676]] 

Y1_val shape: (4, 5) 
 [[-0.9999996   0.9999997   0.6937082   0.9998783  -0.97224885]
 [-0.99849474  0.9996176  -0.82427216  0.9761792  -0.9902969 ]
 [-0.9992558   0.99991703  0.5127921   0.9826173  -0.77418613]
 [-0.98260427  0.9602705   0.865418   -0.37578002  0.92803943]]


In [35]:
# 시간축, 샘플축이 한 array에 있을 경우
tf.reset_default_graph()

# 시간축 상에 RNN cell만들 개수
n_steps = 2
n_inputs = 3
n_neurons = 5

# shape=[데이터개수, n_steps, n_inputs]
X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])

# 0:샘플축, 1:시간축 2:요소축 순서를 시간축, 샘플축, 요소축으로 바꾸기
X_seqs = tf.unstack(tf.transpose(X, perm=[1,0,2]))

# 위에서 전처리한 데이터넣기 위한 RNN cell정의
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# RNN 모두 연결시키기
outputs_seqs, states = tf.nn.static_rnn(cell=basic_cell,
                                  inputs=X_seqs,
                                  dtype=tf.float32)
outputs = tf.transpose(tf.stack(outputs_seqs), perm=[1,0,2])

X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [3,4,5]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_seqs_val, outputs = sess.run([outputs_seqs, outputs], feed_dict={X: X_batch})
    print('Before Transpose(시간축 기준으로 나눔):',np.array(outputs_seqs_val).shape,'\n',np.array(outputs_seqs_val))
    print('-'*60)
    print('After Transpose(동일한 샘플 넘버기준으로 나눔):', outputs.shape,'\n', outputs)

Before Transpose(시간축 기준으로 나눔): (2, 4, 5) 
 [[[ 0.7588106  -0.6098711   0.68830967 -0.8216917   0.7818271 ]
  [ 0.7124619  -0.99700797  0.25520456 -0.99986917  0.998569  ]
  [ 0.6589339  -0.99998146 -0.31202933 -0.9999999   0.9999916 ]
  [-0.9977034  -0.99639654 -0.99764323 -0.99999106  0.29535118]]

 [[-0.94423974 -0.99999976 -0.9870186  -1.          0.9999986 ]
  [ 0.60949427 -0.9997506   0.62871623 -0.99988776  0.9996657 ]
  [-0.84633607 -0.9999709  -0.877157   -0.9999917   0.9999124 ]
  [-0.29112646 -0.98232144 -0.4156617  -0.94314075  0.9825436 ]]]
------------------------------------------------------------
After Transpose(동일한 샘플 넘버기준으로 나눔): (4, 2, 5) 
 [[[ 0.7588106  -0.6098711   0.68830967 -0.8216917   0.7818271 ]
  [-0.94423974 -0.99999976 -0.9870186  -1.          0.9999986 ]]

 [[ 0.7124619  -0.99700797  0.25520456 -0.99986917  0.998569  ]
  [ 0.60949427 -0.9997506   0.62871623 -0.99988776  0.9996657 ]]

 [[ 0.6589339  -0.99998146 -0.31202933 -0.9999999   0.9999916 ]
  [-0.846

In [ ]:
## 축 변경 비교 예시 ##

In [11]:
# 시간축을 바깥으로 변경한 예시
X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [3,4,5]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

tf.Session().run(X_seqs, feed_dict={X: X_batch})

array([[[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.],
        [9., 0., 1.]],

       [[9., 8., 7.],
        [3., 4., 5.],
        [6., 5., 4.],
        [3., 2., 1.]]], dtype=float32)

In [12]:
# 한 시간축에 t=0,1모두 다들어가 있음->이런식으로 RNN cell에 넣어선 안됨!
X_batch

array([[[0, 1, 2],
        [9, 8, 7]],

       [[3, 4, 5],
        [3, 4, 5]],

       [[6, 7, 8],
        [6, 5, 4]],

       [[9, 0, 1],
        [3, 2, 1]]])

In [16]:
# unstack한 예시
X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [3,4,5]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

tf.Session().run(X_seqs, feed_dict={X: X_batch})

[array([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.],
        [9., 0., 1.]], dtype=float32),
 array([[9., 8., 7.],
        [3., 4., 5.],
        [6., 5., 4.],
        [3., 2., 1.]], dtype=float32)]

In [37]:
# dynamic_rnn 으로 입력값 unstack+static_rnn + 결과값 stack 한 번에!
tf.reset_default_graph()

n_steps = 2
n_inputs = 3
n_neurons = 5

# 입력데이터 shape을 가장 안쪽은 입력데이터개수, 그다음은 시간축, 그다음은 Sample개수로 들어오도록 설정
X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])

# RNN cell 설정
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# RNN cell 이어붙이기
outputs_seqs, states = tf.nn.dynamic_rnn(cell=basic_cell,
                                        inputs=X, dtype=tf.float32)

X_batch = np.array([
    [[0,1,2], [9,8,7]],
    [[3,4,5], [3,4,5]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_seqs_val, final_states = sess.run([outputs_seqs,
                                              states],
                                             feed_dict={X: X_batch})

    print("Sequence of Outputs:\n", outputs_seqs_val)
    print()
    # 최종 state는 t=1일 때의 모든 Sample에 대한 중간출력값이랑 동일!
    print("Final Hidden state:\n", final_states)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Sequence of Outputs:
 [[[ 0.1723842   0.8854139   0.6285169  -0.9133235   0.6663468 ]
  [-0.69875354  1.          0.99843043 -1.          0.8548731 ]]

 [[ 0.02909221  0.9999394   0.98397803 -0.9999588   0.8477142 ]
  [-0.3728183   0.99970233  0.9392211  -0.99992913  0.97949547]]

 [[-0.11540549  1.          0.9994284  -1.          0.9343763 ]
  [-0.7477404   0.9999533   0.95829374 -0.9999859   0.81684226]]

 [[-0.9996545   0.99882543 -0.30663958 -0.9809706  -0.88656986]
  [-0.6659823   0.98878616  0.6566779  -0.9628897  -0.8913977 ]]]

Final Hidden state:
 [[-0.69875354  1.          0.99843043 -1.          0.8548731 ]
 [-0.3728183   0.99970233  0.9392211  -0.99992913  0.97949547]
 [-0.7477404   0.9999533   0.95829374 -0.9999859   0.81684226]
 [-0.6659823   0.98878616  0.6566779  -0.9628897  -0.8913977 ]]


In [39]:
# 데이터의 가변적으로 설정해 입력할 수 있는 dynamic_rnn!
tf.reset_default_graph()

n_steps = 2
n_inputs = 3
n_neurons = 5

X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])
# 데이터 가변적인 길이 array 담을 placeholder 정의 - 길이는 정수로 설정
seq_length = tf.placeholder(tf.int8, shape=[None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs_seqs, states = tf.nn.dynamic_rnn(cell=basic_cell,
                                        inputs=X,
                                        dtype=tf.float32)
X_batch = np.array([
    # t=0      t=1
    [[0,1,2], [9,8,7]],
    [[3,4,5], [0,0,0]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

# 변경할 데이터 길이 정의
seq_length_batch = np.array([2, 1, 2, 2])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val, states_val = sess.run([outputs_seqs, states],
                                      feed_dict={X: X_batch})

print(outputs_val.shape, '\n', outputs_val)
print('-'*70)
print(states_val.shape, '\n', states_val)

(4, 2, 5) 
 [[[-0.12714577  0.69758916 -0.1745883   0.83260804  0.27155182]
  [-0.99996346  0.9999998   0.99999803  0.55163074 -0.5807408 ]]

 [[-0.97063166  0.9996537   0.9522551   0.9549142   0.21481717]
  [ 0.67704976 -0.7807796   0.8640524  -0.8003127  -0.17191263]]

 [[-0.99942654  0.9999997   0.99915975  0.9884212   0.15659471]
  [-0.99473965  0.9999117   0.9999736  -0.73771286 -0.54344183]]

 [[-0.9996019   0.999168    0.99997425 -0.99981093 -0.97119194]
  [-0.96100426  0.99427086  0.9944113   0.01406603 -0.6689392 ]]]
----------------------------------------------------------------------
(4, 5) 
 [[-0.99996346  0.9999998   0.99999803  0.55163074 -0.5807408 ]
 [ 0.67704976 -0.7807796   0.8640524  -0.8003127  -0.17191263]
 [-0.99473965  0.9999117   0.9999736  -0.73771286 -0.54344183]
 [-0.96100426  0.99427086  0.9944113   0.01406603 -0.6689392 ]]


In [40]:
# 데이터의 가변적으로 설정해 입력할 수 있는 dynamic_rnn!
tf.reset_default_graph()

n_steps = 2
n_inputs = 3
n_neurons = 5

X = tf.placeholder(tf.float32, shape=[None, n_steps, n_inputs])
# 데이터 가변적인 길이 array 담을 placeholder 정의 - 길이는 정수로 설정
seq_length = tf.placeholder(tf.int8, shape=[None])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs_seqs, states = tf.nn.dynamic_rnn(cell=basic_cell,
                                        inputs=X,
                                        dtype=tf.float32)
X_batch = np.array([
    # t=0      t=1
    [[0,1,2], [9,8,7]],
    [[3,4,5], [0,0,0]],
    [[6,7,8], [6,5,4]],
    [[9,0,1], [3,2,1]]
])

# 변경할 데이터 길이 정의 (이전과 다르게 설정)
# 데이터 길이를 변경한다는 것은 0을 넣는다는 뜻!
seq_length_batch = np.array([2, 0, 2, 2])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val, states_val = sess.run([outputs_seqs, states],
                                      feed_dict={X: X_batch})

print(outputs_val.shape, '\n', outputs_val)
print('-'*70)
print(states_val.shape, '\n', states_val)

(4, 2, 5) 
 [[[-0.45214415  0.3666158  -0.7428127   0.69905645 -0.8225881 ]
  [-0.9997777  -0.8702889  -0.9999999   0.791732   -1.        ]]

 [[-0.9786259   0.02357117 -0.998974    0.8794445  -0.9998667 ]
  [-0.11278764  0.14909352 -0.91579825  0.6067078   0.66673255]]

 [[-0.9993814  -0.3251154  -0.9999965   0.9545958  -0.9999999 ]
  [-0.99778044 -0.8028292  -0.9999957   0.715916   -0.9999528 ]]

 [[-0.9923134  -0.9997628  -0.9999413  -0.8921898  -0.99994826]
  [-0.9639333  -0.8127727  -0.9982045  -0.62882626 -0.8561248 ]]]
----------------------------------------------------------------------
(4, 5) 
 [[-0.9997777  -0.8702889  -0.9999999   0.791732   -1.        ]
 [-0.11278764  0.14909352 -0.91579825  0.6067078   0.66673255]
 [-0.99778044 -0.8028292  -0.9999957   0.715916   -0.9999528 ]
 [-0.9639333  -0.8127727  -0.9982045  -0.62882626 -0.8561248 ]]


In [8]:
# Case 1-1
tf.reset_default_graph()

n_neurons = 2

# RNN cell
cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
x_data = np.array([[[1,0,0,0]]], dtype=np.float32)
outputs, state = tf.nn.dynamic_rnn(cell=cell, inputs=x_data,
                                  dtype=np.float32)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val = outputs.eval()
    print(outputs_val)

[[[ 0.37729472 -0.6199186 ]]]


In [4]:
# Case 1-2
tf.reset_default_graph()

n_neurons = 2
n_inputs = 4

X = tf.placeholder(tf.float32, shape=[None, 1, n_inputs])

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, state = tf.nn.dynamic_rnn(cell=basic_cell,
                                  inputs=X, dtype=tf.float32)
X_batch = np.array([
    [[1,0,0,0]]
])

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    outputs_val = sess.run([outputs], feed_dict={X: X_batch})
    print(outputs_val)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[array([[[-0.5861949,  0.6538274]]], dtype=float32)]


In [3]:
# Case 2 - 'Hello' String RNN example
# h,e,l,l을 RNN에 넣어서 문자열 'o'가 나올 것을 예측해보자
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

n_neurons = 4
n_inputs = 4
#
batch_size = 1
# RNN cell 개수
sequence_length = 4

# hello 문자열 해당 인덱스 번호
idx2char = ['h','e','l','l','o']
## h,e,l,l 까지 데이터로 나타내기
# ex. 0번 = idx2char의 0번 index = 'h'
x_data = [[0,1,2,2]]
x_data_ohe = [[[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,1,0]]]

# 중간출력 예측값에 대한 데이터 정답 1=e, 2=l, 3=o ->이거와 비교를 통해 loss를 감소시킬 것임!
y_data = [[1,2,2,3]]
    

X = tf.placeholder(tf.float32, shape=[None, sequence_length, n_inputs])
Y = tf.placeholder(tf.int32, shape=[None, sequence_length])

cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
# 다음 RNN Cell에서 중간출력값을 만들기 위해 이전 RNN cell에서 전달할 초기 state(Wh)를 0값으로 설정
# state는 RNN cell안에서 neuron에서 나오기 때문에 size가 n_neurons사이즈와 동일하게 나옴
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, states = tf.nn.dynamic_rnn(cell=cell, inputs=X,
                                   initial_state=initial_state,
                                   dtype=tf.float32)
# 각 중간출력 y값들의 Loss를 종합할 때 곱할 Weight(1로 설정)
weights = tf.ones([batch_size, sequence_length])
# 각 중간출력 y값들에 대한 loss summation
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs,
                                                targets=Y,
                                                weights=weights)
loss = tf.reduce_mean(sequence_loss)

# SGD
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(1001):
        l, _ = sess.run([loss, train],
                          feed_dict={X: x_data_ohe,
                                    Y: y_data})
        # Tensor이름이랑 변수이름 같게 하지 말기..에러발생..
        predict = sess.run(prediction,
                             feed_dict={X: x_data_ohe})
        print('Y_hat:', predict, 'Y:', y_data)


Y_hat: [[1 3 1 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 1 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 1 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 2]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y

Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y

Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y

Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y_hat: [[1 2 2 3]] Y: [[1, 2, 2, 3]]
Y